In [1]:
#dependencies

import numpy as np #vectorization
import random #Distrubution:generate text
import tensorflow as tf #ML
import datetime #clockTrainTime

/home/abhiraj/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
text = open('dataset/wikitext-103-raw/wiki.test.raw').read()
print('text len (in chars)', len(text))

print('head of text')
print(text[:1000])

text len (in chars) 1288556
head of text
 
 = Robert Boulter = 
 
 Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy 's Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . 
 In 2006 , Boulter starred alongside Whishaw in the play Citizenship writte

In [3]:
#print out our chars and sort them
chars = sorted(list(set(text)))
char_size = len(chars)
print('number of chars', char_size)
print(chars)

number of chars 259
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '¥', '©', '°', '½', 'Á', 'Æ', 'É', '×', 'ß', 'à', 'á', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'î', 'ñ', 'ó', 'ô', 'ö', 'ú', 'ü', 'ć', 'č', 'ě', 'ī', 'ł', 'Ō', 'ō', 'Š', 'ū', 'ž', 'ǐ', 'ǔ', 'ǜ', 'ə', 'ɛ', 'ɪ', 'ʊ', 'ˈ', 'ː', '̍', '͘', 'Π', 'Ω', 'έ', 'α', 'β', 'δ', 'ε', 'ι', 'λ', 'μ', 'ν', 'ο', 'π', 'ς', 'σ', 'τ', 'υ', 'ω', 'ό', 'П', 'в', 'д', 'и', 'к', 'н', 'א', 'ב', 'י', 'ל', 'ר', 'ש', 'ת', 'ا', 'ت', 'د', 'س', 'ك', 'ل', 'و', 'ڠ', 'ग', 'न', 'र', 'ल', 'ष', 'ु', 'े', 'ो', '्', 'ả', 'ẩ', '‑', '–', '—', '’', '“', '”'

In [4]:
char2id = dict((c,i) for i,c in enumerate(chars))
id2char = dict((i,c) for i,c in enumerate(chars))

In [5]:
#generate probability of each next character
def sample(prediction):
  r = random.uniform(0,1)
  #store prediction character
  s = 0
  char_id = len(prediction) - 1
  
  for i in range(len(prediction)): 
    s += prediction[i]
    if s >= r:
      char_id = i
      break
      
  char_one_hot = np.zeros(shape[char_size])
  char_one_hot[char_id] = 1.0
  return char_one_hot

#### DRY RUN: SAMPLE 


In [6]:
prediction = 'abhiraj'

In [7]:
r = random.uniform(0,1)
r

0.24453480046817844

In [8]:
s = 0
char_id = len(prediction) - 1
char_id

6

In [9]:
for i in range(len(prediction)): 
  s += prediction[i]
  print(s)
  if s >= r:
    char_id = i
    break

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

#### ----- DONE: DRY RUN: SAMPLE -----

In [ ]:
#vectorize data

len_per_section = 50
skip = 2 # Ideally should be 50 but due to lack of data use 2
sections = []
next_chars = []

for i in range(0, len(text) - len_per_section, skip):
  sections.append(text[i: i + len_per_section])
  next_chars.append(text[i + len_per_section])
  
#Vectorize
X = np.zeros((len(sections), len_per_section, char_size), dtype = np.float32)

y = np.zeros((len(sections), char_size), dtype = np.float32)

#for ecah char in each sentence convert each char to an ID
#for each section convert the labels to ids
for i,section in enumerate(sections):
  for j, char in enumerate(section):
    X[i,j, char2id[char]] == 1
  y[i, char2id[next_chars[i]]] == 1

In [ ]:
#machine learning

batch_size = 512
max_steps = 72000
log_every = 100
save_every = 6000
hidden_nodes = 1024

test_start = 'I am thinking that'

#checkpointing
checkpoint_directory = 'checkpoints/wiki'

if tf.gfile.Exists(checkpoint_directory):
  tf.gfile.DeleteRecursively(checkpoint_directory)
tf.gfile.MakeDirs(checkpoint_directory)

print('training data size:', len(X))
print('approximate steps per ephoch:', int(len(X)/batch_size))

In [10]:
#build model
graph = tf.Graph()
with graph.as_default():
  
  global_step = tf.Variable(0)
  data = tf.placeholder(tf.float32, [batch_size, len_per_section, char_size])
  labels = tf.placeholder(tf.float32, [batch_size, char_size])
  
  #input gate, output gate, forget gate, internal state
  #They will be calculated in vaccums
  
  #Input
  w_ii = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  w_io = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  b_i = tf.Variable(tf.zeros([1, hidden_nodes]))
  
  #Forget
  w_fi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  w_fo = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  b_f = tf.Variable(tf.zeros([1, hidden_nodes]))
  
  #Output
  w_oi = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  w_oo = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  b_o = tf.Variable(tf.zeros([1, hidden_nodes]))
  
  #Memory cell/Hidden
  w_ci = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  w_co = tf.Variable(tf.truncated_normal([char_size, hidden_nodes], -0.1, 0.1))
  b_c = tf.Variable(tf.zeros([1, hidden_nodes]))
  
  def lstm(i, o, state):
    input_gate = tf.sigmoid(tf.matmul(i ,w_ii) + tf.matmul(o, w_io) + b_i)
    
    forget_gate = tf.sigmoid(tf.matmul(i ,w_fi) + tf.matmul(o, w_fo) + b_f)
    
    output_gate = tf.sigmoid(tf.matmul(i, w_oi) + tf.matmul(o, w_oo) + b_o)
    
    memory_cell = tf.sigmoid(tf.matmul(i, w_ci) + tf.matmul(o, w_co) + b_c)
    
    state = forget_gate * state + input_get * memory_cell
    
    output = output_gate * tf.tanh(state)
    
    return output, state
    

  #Operation
  output = tf.zeros([batch_size, hidden_nodes])
  state = tf.zeros([batch_size, hidden_nodes])
  
  #unrolled lstm loop for each i/p set
  for i in range(len_per_section):
    output, state = lstm(data[:, i, :], output, state)
    if i == 0:
      outputs_all_i = output
      labels_all_i = data[:, i+1, :]
      
    elif i != len_per_section -1:
      outputs_all_i = tf.concat(0, [outputs_all_i, output])
      labels_all_i = tf.concat(0, [labels_all_i, data[:, i+1, :]])
      
    else:
      outputs_all_i = tf.concat(o, [outputs_all_i, output])
      labels_all_i = tf.concat(0, [labels_all_i, labels])
  
  #Classification
  w = tf.Variable(tf.truncated_normal([hidden_nodes, char_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([char_size]))
  
  logits = tf.matmul(outputs_all_i, w) + b
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels_all_i))
  
  optimizer = tf.train.GradientDescentOptimizer(10.).minimize(loss, global_step=global_step)
    
#   test_data = tf.placeholder(tf.float32, shape = [1, char_size])
#   test_output = tf.Variable(tf.zeros([1, hidden_nodes]))
#   test_state = tf.Variables(tf.zeros([1, hidden_nodes]))
  
#   reset_test_state = tf.group(test_output.assign(tf.zeros([1, hidden_nodes])), 
#                                 test_state.assign(tf.zeros([1, hidden_nodes])))

# #   LSTM
#   test_output, test_state = lstm(test_data, test_output, test_state)
#   test_prediction = tf.nn.softmax(tf.matmul(test_output, w) + b)
    

NameError: name 'batch_size' is not defined